<a href="https://colab.research.google.com/github/jhenningsen/Equity_Analysis/blob/main/SMA_Short_VIX_Adjust.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##SMA_Short_VIX_Adjust
---
This notebook analyzes the historical price data of the ticker to evaluate the impact of **Simple Moving Average (SMA)** sign changes on asset returns.

* It begins by downloading historical data via the `yfinance` library and cleaning the resulting DataFrame into a flat table format.
* A custom function calculates the SMA for a given window and identifies "sign changes," which occur when the closing price crosses above or below the moving average.
* The analysis specifically filters for instances where the price drops below the SMA (sign change to -1) and calculates the subsequent return between those events.
* The script iterates through a range of lookback periods (from 3 to 21) to aggregate these returns by year, allowing for a comparison of how different SMA lengths perform over time.
* The final output is a concatenated DataFrame that summarizes the total yearly returns triggered by these SMA crossovers for each lookback value.

---

In [23]:
# Import necessary libraries
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

# Define a ticker and a date range for your data
ticker = 'UVXY'
start_date = '2012-01-01'
end_date = '2025-12-25'
window = 5
lookback_range = [3,21]
vix_limit = 18

# Download historical data from Yahoo Finance for a single ticker.
# This will result in a DataFrame with 'Date' as a simple index.
data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True)

# Use reset_index() to convert the 'Date' index into a column.
data = data.reset_index()

# Now, to get a new DataFrame with just the 'Price' level, we can use droplevel()
# This removes the 'Ticker' level from the columns, leaving only the 'Price' level.
data = data.droplevel(level='Ticker', axis=1)

# Now, add the 'Ticker' column at position 1 (right after the 'Date' column).
data.insert(1, 'Ticker', ticker)

# The DataFrame is now a flat table with no MultiIndex.
display(data)

[*********************100%***********************]  1 of 1 completed


Price,Date,Ticker,Close,High,Low,Open,Volume
0,2012-01-03,UVXY,1.632000e+11,1.675500e+11,1.612500e+11,1.656000e+11,0
1,2012-01-04,UVXY,1.569000e+11,1.705500e+11,1.564500e+11,1.678500e+11,0
2,2012-01-05,UVXY,1.497000e+11,1.642500e+11,1.492500e+11,1.611000e+11,0
3,2012-01-06,UVXY,1.449000e+11,1.530000e+11,1.419000e+11,1.471500e+11,0
4,2012-01-09,UVXY,1.405500e+11,1.446000e+11,1.390500e+11,1.422000e+11,0
...,...,...,...,...,...,...,...
3511,2025-12-18,UVXY,4.108000e+01,4.255000e+01,4.060000e+01,4.165000e+01,10864500
3512,2025-12-19,UVXY,3.866000e+01,4.045000e+01,3.853000e+01,4.040000e+01,6814300
3513,2025-12-22,UVXY,3.680000e+01,3.784000e+01,3.667000e+01,3.754000e+01,5383700
3514,2025-12-23,UVXY,3.698000e+01,3.716000e+01,3.661000e+01,3.708000e+01,4483100


In [24]:
# Download historical data from Yahoo Finance for a single ticker.
# This will result in a DataFrame with 'Date' as a simple index.
vix_ticker = '^VIX'
vix_data = yf.download(vix_ticker, start=start_date, end=end_date, auto_adjust=True)

# Use reset_index() to convert the 'Date' index into a column.
vix_data = vix_data.reset_index()

# Now, to get a new DataFrame with just the 'Price' level, we can use droplevel()
# This removes the 'Ticker' level from the columns, leaving only the 'Price' level.
vix_data = vix_data.droplevel(level='Ticker', axis=1)

# Now, add the 'Ticker' column at position 1 (right after the 'Date' column).
vix_data.insert(1, 'Ticker', vix_ticker)

# The DataFrame is now a flat table with no MultiIndex.
display(vix_data)

[*********************100%***********************]  1 of 1 completed


Price,Date,Ticker,Close,High,Low,Open,Volume
0,2012-01-03,^VIX,22.969999,23.100000,22.540001,22.950001,0
1,2012-01-04,^VIX,22.219999,23.730000,22.219999,23.440001,0
2,2012-01-05,^VIX,21.480000,23.090000,21.340000,22.750000,0
3,2012-01-06,^VIX,20.629999,21.719999,20.580000,21.240000,0
4,2012-01-09,^VIX,21.070000,21.780001,21.000000,21.670000,0
...,...,...,...,...,...,...,...
3511,2025-12-18,^VIX,16.870001,17.680000,15.930000,17.610001,0
3512,2025-12-19,^VIX,14.910000,16.530001,14.910000,16.309999,0
3513,2025-12-22,^VIX,14.080000,15.260000,14.030000,15.160000,0
3514,2025-12-23,^VIX,14.000000,14.450000,13.640000,14.090000,0


In [25]:
def calculate_sma(df, lookback):
    """
    Calculates a simple moving average for a DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame with a 'close' column.
        lookback (int): The number of periods for the moving average.

    Returns:
        pd.DataFrame: The DataFrame with a new column for the moving average.
    """
    # Create a copy to avoid modifying the original DataFrame
    df_sma = df.copy()

    # Calculate the simple moving average
    df_sma['SMA'] = df_sma['Close'].rolling(window=lookback).mean()

    # Calculate the difference between the SMA and the Close price
    df_sma['SMA_sign'] = np.sign(df_sma['Close'] - df_sma['SMA'])

    return df_sma



In [26]:
def analyze_sma_changes(df, vix_df, lookback, vix_threshold):
    """
    Analyzes the impact of Simple Moving Average bearish sign changes on price differences.

    Args:
        df (pd.DataFrame): The input DataFrame with historical price data.
        lookback (int): The number of periods for the moving average.

    Returns:
        pd.DataFrame: A DataFrame containing the sum of Next_Close_Diff at SMA sign changes by year,
                      including the lookback value.
    """

    # 1. Calculate SMA and signs for every day
    df_with_sma = calculate_sma(df=df.copy(), lookback=lookback)

    # 2. Merge VIX data into the main dataframe based on Date
    # We use 'left' join to keep all price dates and pull in the corresponding VIX close
    vix_subset = vix_df[['Date', 'Close']].rename(columns={'Close': 'VIX_Close'})
    df_with_sma = pd.merge(df_with_sma, vix_subset, on='Date', how='left')

    # 3. Identify ALL sign changes (crosses up AND crosses down)
    previous_sign = df_with_sma['SMA_sign'].shift(1)
    all_crosses_mask = (df_with_sma['SMA_sign'] != previous_sign) & (~previous_sign.isna())

    # Create a DataFrame of every signal event
    all_signals = df_with_sma[all_crosses_mask].copy()

    # 4. Calculate Bearish Return: (Entry Price - Exit Price) / Entry Price
    # Because all_signals contains the NEXT signal (the Bullish cross),
    # shift(-1) now correctly points to the "Exit".
    all_signals.loc[:, 'Next_Close_Return'] = (
        (all_signals['Close'] - all_signals['Close'].shift(-1)) / all_signals['Close']
    )

    # 5. FILTER FOR BEARISH ENTRIES + VIX THRESHOLD
    # - SMA_sign == -1 (Price crossed below SMA)
    # - VIX_Close > vix_threshold (Volatility is high enough)
    bearish_trades = all_signals[
        (all_signals['SMA_sign'] == -1) &
        (all_signals['VIX_Close'] > vix_threshold)
    ].copy()

    # 6. Clean up and Aggregate
    bearish_trades['Date'] = pd.to_datetime(bearish_trades['Date'])
    bearish_trades['Year'] = bearish_trades['Date'].dt.year

    yearly_results = bearish_trades.groupby('Year')['Next_Close_Return'].sum().reset_index()
    yearly_results['Lookback'] = lookback
    yearly_results['VIX_Threshold'] = vix_threshold

    return yearly_results[['Year', 'Lookback', 'VIX_Threshold','Next_Close_Return']]

In [29]:
# Initialize an empty list to store the results from each lookback value
results_list = []

# Iterate through each value in the specified range
for lookback_value in range(lookback_range[0], lookback_range[1] + 1):
    # Calculate SMA changes for the current lookback value
    #df_sma_result = analyze_sma_changes(df=data.copy(), lookback=lookback_value)
    df_sma_result = analyze_sma_changes(df=data.copy(), vix_df=vix_data, lookback=lookback_value, vix_threshold=vix_limit)
     # Append the result to the list
    results_list.append(df_sma_result)

# Concatenate all the DataFrames in the list into a single DataFrame
all_sma_results = pd.concat(results_list, ignore_index=True)

# Print the resulting DataFrame
print("DataFrame with Simple Moving Average analysis for different lookback values:")
display(all_sma_results)

DataFrame with Simple Moving Average analysis for different lookback values:


,Year,Lookback,VIX_Threshold,Next_Close_Return
0,2012,3,18,1.518477
1,2013,3,18,0.604204
2,2014,3,18,0.329355
3,2015,3,18,0.585802
4,2016,3,18,0.441661
...,...,...,...,...
217,2021,21,18,0.275146
218,2022,21,18,-0.193160
219,2023,21,18,0.496538
220,2024,21,18,0.071249


In [30]:
# Pivot the DataFrame
pivoted_sma_results = all_sma_results.pivot(index='Year', columns='Lookback', values='Next_Close_Return')

# Display the pivoted DataFrame
print("Pivoted DataFrame with Lookback as columns, Year as rows, and Next_Close_Return as values:")
display(pivoted_sma_results)

Pivoted DataFrame with Lookback as columns, Year as rows, and Next_Close_Return as values:


Lookback,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
Year,,,,,,,,,,,,,,,,,,,
2012,1.518477,1.462070,1.363585,1.278909,1.809015,1.662597,1.514705,1.799580,1.799580,1.532919,1.479221,1.567675,1.240885,1.240885,1.174492,1.167856,1.182219,1.115338,0.995366
2013,0.604204,0.390902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014,0.329355,0.630764,0.508292,0.597377,0.500460,0.502147,0.357185,0.368850,0.368850,0.380939,0.380939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015,0.585802,0.291696,0.097449,0.099459,0.535460,0.575357,0.660838,0.468062,0.424807,0.289104,0.332204,0.332204,0.215787,0.215787,0.216119,0.051120,-0.038766,-0.038766,0.154010
2016,0.441661,0.938740,0.584311,0.795673,1.094197,1.153251,1.069644,1.083293,0.895151,1.065026,1.146798,1.146145,1.403133,1.483326,1.483326,1.366832,1.459873,1.844973,1.834931
2018,-0.458142,-0.118320,0.181737,-0.167380,0.243398,0.355070,0.133371,-0.058239,-0.065126,-0.169368,-0.073095,-0.073095,-0.266822,-0.164944,0.012119,0.202303,0.177702,0.039680,0.054229
2019,0.235455,0.077292,0.245988,0.738080,0.238133,0.223514,0.277395,0.016837,0.277395,0.318354,0.395295,0.395295,0.395295,0.359668,0.359668,0.359668,0.359668,0.359668,0.336293
2020,0.667361,0.946321,0.944927,0.638175,0.676449,0.257814,0.660689,0.614415,0.711001,0.416517,0.264488,0.248707,0.106742,0.106742,0.389221,0.363408,0.439383,0.337248,0.419053
2021,0.861419,0.483054,0.050046,0.338119,0.690326,0.258527,0.215780,0.164444,0.111512,0.187139,0.227648,0.202243,-0.129734,-0.048742,0.029788,0.029788,0.083084,0.245133,0.275146


In [31]:
# Ensure 'Date' column in the initial 'data' DataFrame is in datetime format
data['Date'] = pd.to_datetime(data['Date'])

# Extract the year from the 'Date' column
data['Year'] = data['Date'].dt.year

# Group by year and get the first and last close prices
yearly_price_change = data.groupby('Year')['Close'].agg(['first', 'last'])

# Calculate the difference between the last and first close price for each year
yearly_price_change['Yearly_Return'] = (yearly_price_change['first'] - yearly_price_change['last']) / yearly_price_change['first']

# Drop the 'first' and 'last' columns
yearly_price_change = yearly_price_change.drop(columns=['first', 'last'])

# Display the result
print("Difference between the last and first close price of each year:")
display(yearly_price_change)

Difference between the last and first close price of each year:


,Yearly_Return
Year,
2012,0.967984
2013,0.895582
2014,0.639478
2015,0.766954
2016,0.945244
2017,0.931933
2018,-0.729735
2019,0.834574
2020,0.122012


In [32]:
# Join the yearly_price_change DataFrame with the yearly_next_close_diff_sum Series on the 'Year' index
comparison_df = yearly_price_change.join(pivoted_sma_results)

# Display the combined DataFrame, excluding the 'first' and 'last' columns
display(comparison_df)

,Yearly_Return,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
Year,,,,,,,,,,,,,,,,,,,,
2012,0.967984,1.518477,1.462070,1.363585,1.278909,1.809015,1.662597,1.514705,1.799580,1.799580,1.532919,1.479221,1.567675,1.240885,1.240885,1.174492,1.167856,1.182219,1.115338,0.995366
2013,0.895582,0.604204,0.390902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014,0.639478,0.329355,0.630764,0.508292,0.597377,0.500460,0.502147,0.357185,0.368850,0.368850,0.380939,0.380939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015,0.766954,0.585802,0.291696,0.097449,0.099459,0.535460,0.575357,0.660838,0.468062,0.424807,0.289104,0.332204,0.332204,0.215787,0.215787,0.216119,0.051120,-0.038766,-0.038766,0.154010
2016,0.945244,0.441661,0.938740,0.584311,0.795673,1.094197,1.153251,1.069644,1.083293,0.895151,1.065026,1.146798,1.146145,1.403133,1.483326,1.483326,1.366832,1.459873,1.844973,1.834931
2017,0.931933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018,-0.729735,-0.458142,-0.118320,0.181737,-0.167380,0.243398,0.355070,0.133371,-0.058239,-0.065126,-0.169368,-0.073095,-0.073095,-0.266822,-0.164944,0.012119,0.202303,0.177702,0.039680,0.054229
2019,0.834574,0.235455,0.077292,0.245988,0.738080,0.238133,0.223514,0.277395,0.016837,0.277395,0.318354,0.395295,0.395295,0.395295,0.359668,0.359668,0.359668,0.359668,0.359668,0.336293
2020,0.122012,0.667361,0.946321,0.944927,0.638175,0.676449,0.257814,0.660689,0.614415,0.711001,0.416517,0.264488,0.248707,0.106742,0.106742,0.389221,0.363408,0.439383,0.337248,0.419053


In [33]:
# Calculate the sum and standard deviation of each column in the comparison_df
grand_totals = comparison_df.sum()
standard_deviations = comparison_df.std()

# Calculate the Mean Absolute Deviation for each column
mean_absolute_deviations = comparison_df.apply(lambda x: (x - x.mean()).abs().mean())

# Combine the metrics into a single DataFrame for display
summary_df = pd.DataFrame({
    'Grand Total': grand_totals,
    'Standard Deviation': standard_deviations,
    'Mean Absolute Deviation': mean_absolute_deviations
})

# Display the summary DataFrame
print("Grand Totals, Standard Deviations, and Mean Absolute Deviations for each column:")
display(summary_df)

Grand Totals, Standard Deviations, and Mean Absolute Deviations for each column:


,Grand Total,Standard Deviation,Mean Absolute Deviation
Yearly_Return,8.729193,0.458569,0.310603
3,5.849811,0.507413,0.330248
4,6.185491,0.457312,0.348275
5,5.712770,0.394876,0.306213
6,6.282242,0.374947,0.287711
7,7.728468,0.439386,0.282305
8,6.775183,0.429305,0.283889
9,5.978051,0.423115,0.318866
10,5.029008,0.550675,0.382159
11,5.170757,0.522353,0.359068
